<h1>Train Neural Network To Estimate Average Strenghts and Variations on the River </h1>
There was an error in the evaluator for average hand strengths in the PokerAI-Agent. We must therefore retrain the mode. The error came about in generating the training data. 
Beyond correcting the error, we want to change the outputs of the model. We want to leave the outputs at 3, but instead of outputting average win and average loss, which are actually identical, we will generate two estimates for deviations - a std for win/loss and a std for split. 

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from keras import backend as K
dataFolder = 'simulatedHands/'#csv-files of simulated hands are stored here
modelsFolder = 'trainedModels/strengthEvaluator/' #neural network models are stored here

<h2>Simulated boards</h2>
We can still use 1.6million simulated hands form the simHandsRiverMore-Data Set. The error was not to be found in the estimates for Hero-wins and losses. The data frame consists of hero hands and boards, as well as probabilities of winning, loosing, or splitting the hand against random opponent hands.

In [2]:
simHandsRiver = pd.read_csv(dataFolder+'simHandsRiverMore.csv')
simHandsRiver.shape

(1600000, 17)

In [6]:
6**10

60466176

In [3]:
(3**10)**2

3486784401

In [3]:
simHandsRiver.head()

,val1,suit1,val2,suit2,f1val,f1suit,f2val,f2suit,f3val,f3suit,tval,tsuit,rval,rsuit,nwin,nloose,nsplit
0,5,2,11,0,4,1,7,0,13,1,11,2,9,0,0.707071,0.262626,0.030303
1,3,0,14,1,8,2,12,1,5,0,7,2,10,2,0.248485,0.710101,0.041414
2,3,0,3,2,10,1,5,3,12,2,5,2,13,0,0.490909,0.508081,0.001010
3,6,0,10,3,13,1,11,2,2,0,13,0,6,2,0.673737,0.265657,0.060606
4,3,1,13,2,12,2,8,0,7,2,11,2,14,0,0.278788,0.663636,0.057576


Let's check how accurate those percentages are, running a few of those lines against all possible villains.

In [4]:
from itertools import combinations
from PokerHandStrengths import compare
#create a deck
deck=[]
for val in range(2,15):
    for suit in range(4):
        deck.append((val,suit))


In [5]:
#function that takes in a row from the dataFrame and creates the list for hands usable in the compare function
def transformHand(row):
    rowInt = row.astype(int)
    hero = [(rowInt[0], rowInt[1]), (rowInt[2],rowInt[3])]
    board = [(rowInt[4], rowInt[5]), (rowInt[6],rowInt[7]),(rowInt[8], rowInt[9]), (rowInt[10],rowInt[11]), (rowInt[12],rowInt[13])]
    return hero + board

In [6]:
#function takes in heros hand and plays against all opponents. Return pwin, ploose, psplit
from math import comb

def playHands(handHero,deck = deck):
    from math import comb
    ncomb = comb(len(deck),2)
    deckRemaining = set(deck)-set(handHero)
    #iterate through all combinations of 2 cards in deckRemaining
    nwin = 0
    nloose = 0
    nsplit = 0

    for combo in combinations(deckRemaining,2):
        handVillain = list(combo)+handHero[2:]
        
        result = compare({0:handHero, 1: handVillain})
        if result[0]==result[1]:
            nsplit += 1
        elif result[0]<result[1]:
            nwin += 1
        else:
            nloose+=1
    pwin = nwin/ncomb
    ploose = nloose/ncomb
    psplit = nsplit/ncomb
    return pwin, ploose, psplit


Let's check if the percentages in the dataFrame simHandsRiver are the exact ones, or simulated ones.

In [81]:
for i in range(100):
    row = simHandsRiver.iloc[i].astype(int)
    hand = transformHand(row)
    percentages = playHands(hand)
    percentagesArray = np.array(percentages)
    print(percentagesArray - np.array(simHandsRiver.iloc[i,-3:]))

[0.00000000e+00 5.55111512e-17 3.46944695e-18]
[8.32667268e-17 0.00000000e+00 1.38777878e-17]
[0.00000000e+00 0.00000000e+00 1.01915004e-17]
[0.00000000e+00 5.55111512e-17 6.93889390e-18]
[0.00000000e+00 0.00000000e+00 7.63278329e-17]
[2.77555756e-17 0.00000000e+00 3.46944695e-18]
[0.00000000e+00 0.00000000e+00 4.16333634e-17]
[0. 0. 0.]
[5.55111512e-17 0.00000000e+00 7.63278329e-17]
[0.00000000e+00 0.00000000e+00 9.02056208e-17]
[0.00000000e+00 2.77555756e-17 6.07153217e-17]
[5.55111512e-17 0.00000000e+00 9.02056208e-17]
[1.11022302e-16 3.12250226e-17 0.00000000e+00]
[5.55111512e-17 0.00000000e+00 3.46944695e-17]
[0. 0. 0.]
[5.55111512e-17 0.00000000e+00 9.02056208e-17]
[5.55111512e-17 0.00000000e+00 6.07153217e-17]
[0.00000000e+00 0.00000000e+00 6.07153217e-17]
[1.11022302e-16 4.16333634e-17 4.07660017e-17]
[0.00000000e+00 6.24500451e-17 8.32667268e-17]
[8.32667268e-17 0.00000000e+00 3.46944695e-17]
[1.38777878e-17 0.00000000e+00 9.02056208e-17]
[0.00000000e+00 5.55111512e-17 0.00000

We see that the difference between the values in the data frame and the exact values are basically equal, because the difference is almost 0. That means we have perfect values.

We already have a trained neural network that predicts wins and losses on a board for a given hand. It's accuracy is not perfect yet, so we want to improve it, by training more epochs. First let's split our dataframe into train and test. Because it is so large, we can make test-size quite small.

In [7]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(simHandsRiver, test_size=0.01)

In [8]:
X_train = np.array(train.iloc[:,0:-3])
X_test = np.array(test.iloc[:,0:-3])
y_train = np.array(train.iloc[:,-3:-1])
y_test = np.array(test.iloc[:,-3:-1])

Let's load and test our models accuracy and see if we can increase the accuracy with more training. We aim at a loss below 0.001.

In [9]:
networkRiver = load_model(modelsFolder+'strengthRiver.h5')

2022-01-07 17:51:41.265603: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-01-07 17:51:41.265680: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-01-07 17:51:41.265720: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cromi-Lenovo-V15-ADA): /proc/driver/nvidia/version does not exist
2022-01-07 17:51:41.266089: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
networkRiver.evaluate(X_test, y_test)

500/500 [==============================] - 3s 5ms/step - loss: 0.0036


0.00360684166662395

In [ ]:
networkRiver.fit(X_train, y_train, batch_size=16, epochs=20)

In [147]:
prediction = networkRiver.predict(X_test)

In [153]:
prediction[:,1].mean()

0.47252667

<h2>Simulating average hand strengths for training data </h2>
Now that we have a propper network to evaluate individual hand strengths, let's generate training data for average hand strengths. Instead of providing one single hand for hero, we will use all possible combinations, estimate win and loose percentages. Win and loose should be equal. This will also be our sanity check. Because the win and loose probabilities are estimated by a network, they will have some error. So we will use the mean of both as estimator for the strength of average hands. Win-loose- and split percenteges are euqivalent, because they can be calculated from each other. 
Beyond the avg-handStrength, we are interested in a variance meassure of hand strengths. So we will calculate standard deviations for win, and split-pots. The output will therefore be an average value for win/loos, an std for win/loss and a std for splits.

In [11]:
#Function takes in a row from the data frame and calculates pwin/ploose, stdwin, stdloose

def strengthVillain(row):
    import numpy as np
    heroHand = transformHand(row)
    deckRemaining = set(deck) -set(heroHand)
    handsVillain= []
    for combo in combinations(deckRemaining, 2):
        villainHole = list(combo)
        villainHand = np.array(villainHole + heroHand[2:])
        handsVillain.append(list(villainHand.flatten()))
    lstResults=[]
    for i in range(len(handsVillain)):

        rowNew = np.array(handsVillain[i])
        newHand = transformHand(rowNew)
        percentages = playHands(handHero = newHand, deck = deckRemaining)
        percentagesArray = np.array(percentages)

        lstResults.append(percentagesArray)
    pwin = np.array(lstResults)[:,0].mean()
    ploose = np.array(lstResults)[:,1].mean()
    psplit = np.array(lstResults)[:,2].mean()
    stdwin = np.array(lstResults)[:,0].std()
    stdsplit = np.array(lstResults)[:,2].std()
    return pwin, ploose, psplit, stdwin, stdsplit
        

To increase speed, we can calculate handstrengths on multiple threads simulataneously.

In [12]:
#create a cluster of threads and start
nThreads = 8
import ipyparallel as ipp
cluster = ipp.Cluster(n=nThreads)
await cluster.start_cluster()

Starting 8 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


<Cluster(cluster_id='1641574317-it76', profile='default', controller=<running>, engine_sets=['1641574318'])>

In [13]:
rc = cluster.connect_client_sync()
rc.wait_for_engines(n=8)

100%|█████████████████████████████████████████| 8/8 [00:07<00:00,  1.03engine/s]


In [14]:


#to use the map function, we need a function to run, with the number of instances/ threads as argument.
def simulateHands(instance):
    import numpy as np
    resultList = []
    for i in range(len(data)):
        results = strengthVillain(data.iloc[i])
        resultList.append(np.array(results))
    return resultList

In [ ]:
startIndex = 32
nThreads = 8
rowsPerThread = 20
for i in range(nThreads):
    #each thread has to recreate the used objects. Let's load all that we need into the threads.
    rc[i]['data'] = simHandsRiver.iloc[startIndex:].iloc[i*rowsPerThread:i*rowsPerThread+rowsPerThread]
    rc[i]['strengthVillain']=strengthVillain
    rc[i]['transformHand']=transformHand
    rc[i]['playHands'] = playHands
    rc[i]['deck'] = deck
    rc[i]['combinations'] = combinations
    rc[i]['compare']=compare
results = rc[:].map_sync(simulateHands,range(nThreads))
strengthVil = pd.DataFrame(columns = ['pwin', 'ploose', 'psplit', 'stdwin', 'stdloose'])
for i in range(len(results)):
    strengthVil = strengthVil.append(pd.DataFrame(results[i], columns=strengthVil.columns), ignore_index=True)
filename = dataFolder+'VillainStrengthCorrected'+ str(startIndex + len(strengthVil))+'.csv'
strengthVil.to_csv(filename, index= None)
pd.read_csv(filename)
                   

In [29]:
strengthVil = pd.DataFrame(columns = ['pwin', 'ploose', 'psplit', 'stdwin', 'stdloose'])
for i in range(len(results)):
    strengthVil = strengthVil.append(pd.DataFrame(results[i], columns=strengthVil.columns), ignore_index=True)
filename = dataFolder+'VillainStrengthCorrected'+ str(len(strengthVil))+'.csv'
strengthVil.to_csv(filename, index= None)
pd.read_csv(filename)
                   

,pwin,ploose,psplit,stdwin,stdloose
0,0.445246,0.445246,0.021628,0.275155,0.016191
1,0.447849,0.447849,0.016423,0.272910,0.013184
2,0.404062,0.404062,0.103998,0.308995,0.094383
3,0.445872,0.445872,0.020378,0.273302,0.015556
4,0.436589,0.436589,0.038943,0.283147,0.033205
5,0.445100,0.445100,0.021920,0.274837,0.017700
6,0.450698,0.450698,0.010725,0.271046,0.013543
7,0.404430,0.404430,0.103261,0.310291,0.095763
8,0.435896,0.435896,0.040329,0.273899,0.024227
9,0.441831,0.441831,0.028458,0.276170,0.019636


In [17]:
input_dims = 14
fc1_dims = 256
fc2_dims = 128
fc3_dims = 64
fc4_dims = 32
fc5_dims = 16
fc6_dims = 8
fc7_dims = 4



lr = 0.001
modelRiver = keras.Sequential()
modelRiver.add(keras.layers.Dense(fc1_dims, input_dim=input_dims, activation = 'relu'))
modelRiver.add(keras.layers.LeakyReLU (alpha=0.05))
modelRiver.add(keras.layers.Dense(fc2_dims, activation = 'relu'))
modelRiver.add(keras.layers.LeakyReLU (alpha=0.05))
modelRiver.add(keras.layers.Dense(fc3_dims, activation = 'relu'))
modelRiver.add(keras.layers.LeakyReLU (alpha=0.05))
modelRiver.add(keras.layers.Dense(fc4_dims, activation = 'relu'))
modelRiver.add(keras.layers.LeakyReLU (alpha=0.05))
modelRiver.add(keras.layers.Dense(fc5_dims, activation = 'relu'))
modelRiver.add(keras.layers.LeakyReLU (alpha=0.05))
modelRiver.add(keras.layers.Dense(fc6_dims, activation = 'relu'))
modelRiver.add(keras.layers.LeakyReLU (alpha=0.05))
modelRiver.add(keras.layers.Dense(fc7_dims, activation = 'relu'))
modelRiver.add(keras.layers.LeakyReLU (alpha=0.05))


modelRiver.add(keras.layers.Lambda(lambda x: K.sigmoid(0.1*x)))
modelRiver.add(keras.layers.Dense(2,activation=None))
modelRiver.compile(optimizer=Adam(learning_rate=lr), loss='mean_squared_error')